In [1]:
#Packages used

#pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org pandas as pd -vvv
#pip install openpyxl
#python -m  py install dash
#pip install dash-bootstrap-components
#pip install numpy

In [1]:
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, callback, dash_table, State
from datetime import datetime as dt
from datetime import timedelta
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import random
import plotly.express as px
from plotly.subplots import make_subplots




In [2]:
test_data_new = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'TEST') # Import data from new database
test_data_new.dropna(how='all')
test_data_new.dropna(axis=1, how='all', inplace=True)

test_data_old= pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'OLD') # Import data from old database
test_data_old.dropna(how='all')

test_data_old.dropna(axis=1, how='all', inplace=True)  

total_data = pd.concat([test_data_old, test_data_new], ignore_index=True) # Concatenate databases

#total_data.head(5)

c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
# CALENDAR FROM CELLS AND OPERATORS

cells_calendar = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'CELLS') # Import data from new database
#cells_calendar.head()

operators_calendar = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'OPERATORS') # Import data from new database
#operators_calendar.head()

#display(cells_calendar)

In [5]:
#DECLARE FLUID PARAMETERS

cp_water = 4186 # J.(KG.°C)^(-1)
specific_mass_water = 1000 #kg/m3

cp_oil = 1880 # J.(KG.°C)^(-1)
specific_mass_oil = 900 #kg/m3

cp_glycol = 2533 # J.(KG.°C)^(-1)
specific_mass_glycol = 1040 #kg/m3


In [6]:
#CALCULATE CP, RO PARAMETERS

total_data['PERC.M']= pd.to_numeric(total_data['PERC.M'], errors='coerce')
convertion_flow = 0.06 # convert flow from l/min to m3/s

total_data['CP']= 0 # Declare specific heat J.(KG.°C)^(-1)
total_data['RO']= 0 # Declare specific heat J.(KG.°C)^(-1)

total_data["RO"] = np.where(total_data["FLUIDO.CHILLER"] == 'W', specific_mass_water, 
                            (np.where(total_data["FLUIDO.CHILLER"] == 'O', specific_mass_oil, 
                                      (np.where(total_data["FLUIDO.CHILLER"] == 'E', (((100-total_data["PERC.M"])*specific_mass_water) + (total_data["PERC.M"]*specific_mass_oil))/100,
                                                (np.where(total_data["FLUIDO.CHILLER"] == 'G', (((100-total_data["PERC.M"])*specific_mass_water) + (total_data["PERC.M"]*specific_mass_glycol))/100, 0          
                                      )))))))

total_data["CP"] = np.where(total_data["FLUIDO.CHILLER"] == 'W', cp_water, 
                            (np.where(total_data["FLUIDO.CHILLER"] == 'O', cp_oil, 
                                      (np.where(total_data["FLUIDO.CHILLER"] == 'E', (((100-total_data["PERC.M"])*cp_water) + (total_data["PERC.M"]*cp_oil))/100,
                                                (np.where(total_data["FLUIDO.CHILLER"] == 'G', (((100-total_data["PERC.M"])*cp_water) + (total_data["PERC.M"]*cp_glycol))/100, 0          
                                      )))))))  

#total_data[['FLUIDO.CHILLER','WORK.FLUID','CP','RO', 'PERC.M']].head()


In [7]:
# POWER CALCULATION

convertion_flow = 0.00001666 # convert flow from l/min to m3/s

total_data['DELTA'] = (total_data['T.OUT.F.M_C']-total_data['T.IN.F.M_C']).abs() # [°C]

total_data['CALCULATE.POWER'] = total_data['FLUID.FLOW.L_MIN']*convertion_flow * total_data['CP'] * total_data['DELTA'] # [kW]

#total_data[['P_N','FLUID.FLOW.L_MIN', 'CP', 'DELTA', 'CALCULATE.POWER']].head()

In [8]:
# SETTIME PARAMETERS AS DATETIME

total_data['STIMATION'] =  pd.to_timedelta(total_data['STIMATION'], unit='h')
total_data['DURATION'] =  pd.to_timedelta(total_data['DURATION'], unit='h')

#total_data[['AVAILABILITY', 'DUE.DATE', 'STIMATION', 'START.DATE', 'END.DATE', 'DURATION']].dtypes

In [9]:
# TIME CALCULATION

#SET INTEREST INTERVAL

start_date = dt(2024, 9, 10) # (Y/M/D/H/min...)
end_date = dt(2024, 12, 10) # (Y/M/D/H/min...)

#____________________________________________________________________________________________________________________________________
#EXTRACT BUSSINESS DAYS BETWEEN THE DATES

work_days = pd.bdate_range(start_date, end_date) #Defines number of bussiness days between stard date and end date
#print(len(work_days))

#_____________________________________________________________________________________________________________________________________
#FILTER DATAFRAME BY DATE
mask = (total_data['AVAILABILITY']  >= start_date)  & (total_data['DUE.DATE'] <= end_date)

total_data_filtered_by_date = total_data.loc[mask] # all Data filtered by set date

cell_usage_table = total_data_filtered_by_date[['CELL', 'STIMATION', 'DURATION']] #Time table

#_____________________________________________________________________________________________________________________________________
#CALCULATE TOTAL TIME FOR EACH CELL

cell_hours_usage = cell_usage_table.groupby(['CELL']).sum().reset_index() #Total time grouped by cell

#display(cell_hours_usage)

#_____________________________________________________________________________________________________________________________________
#CELL PREDICTED AND EFFETIVE USAGE

cell_40_daily = timedelta(hours = 10) # Max usage of the cell
cell_60_daily = timedelta(hours=6) # Max usage of the cell
cell_110_daily = timedelta(hours=8) # Max usage of the cell

cell_40_availability = len(work_days)*cell_40_daily #availability in the set period
cell_60_availability = len(work_days)*cell_60_daily #availability in the set period
cell_110_availability = len(work_days)*cell_110_daily #availability in the set period

cell_hours_usage['CELL.AVAILABILITY'] = np.where(cell_hours_usage['CELL'] == 40, cell_40_availability,
                                                 (np.where(cell_hours_usage['CELL'] == 60, cell_60_availability,
                                                           np.where(cell_hours_usage['CELL'] == 110, cell_110_availability,0)   
                                                 )))

cell_hours_usage['STIMATED.USAGE'] = cell_hours_usage['STIMATION']*100/cell_hours_usage['CELL.AVAILABILITY']
cell_hours_usage['EFFETIVE.USAGE'] = cell_hours_usage['DURATION']*100/cell_hours_usage['CELL.AVAILABILITY']
                               
display(cell_hours_usage)

,CELL,STIMATION,DURATION,CELL.AVAILABILITY,STIMATED.USAGE,EFFETIVE.USAGE
0,40,0 days 08:00:00,0 days 00:00:00,27 days 12:00:00,1.212121,0.000000
1,60,1 days 00:00:00,2 days 07:00:00,16 days 12:00:00,6.060606,13.888889
2,110,11 days 16:00:00,4 days 15:48:00,22 days 00:00:00,53.030303,21.174242


In [10]:
#NAV BAR DASHBOARD

NVent_LOGO = "https://www.nvent.com/themes/custom/particle/dist/app-drupal/assets/images/logo-nvent.svg" # Address to NVent logo

row_content = dbc.Row(
                     [  
                        dbc.Col(" ", width="10"),
                        dbc.Col(html.Img(src=NVent_LOGO, height="40px")),
                        dbc.Col(dbc.NavbarBrand("TEXA LAB DASHBOARD") ),
                     ],
                     )

navbar = dbc.Navbar(

                     html.A( 
                            row_content, href="https://www.nvent.com/en-us/"                   
                           ),
                               color="dark",
                               dark=True,
                    )

In [11]:
#DATE SELECTOR 

date_card = dbc.Card(
                        dbc.CardBody(
                                [html.H4("Test date range :", className="card-title"),
                                dcc.DatePickerRange(
                                            id="date_filter",
                                            start_date=total_data["DUE.DATE"].min(),
                                            end_date=total_data["DUE.DATE"].max(),                           
                                            )]
                        
                                )
                        )
 


In [12]:
#CELL SELECTOR MENU

cell_card = dbc.Card(
                dbc.CardBody(
                        [
                        html.H4("Cell:", className="card-title"),
                        dcc.Checklist(
                                
                                    options=[
                                        {"label": "040", "value": 40},
                                        {"label": "060", "value": 60},
                                        {"label": "110", "value": 110},
                                        {"label": "Chiller", "value": "Chiller", "disabled": True},
                                        {"label": "Clima", "value": "Clima", "disabled": True}
                                            ],
                                    value=[110],
                                    id = "selected_cell" 
                                    )
                        ]
                        
                            )
                    )

In [13]:
#STATUS CELL SELECTOR

status_card = dbc.Card(
                dbc.CardBody(
                        [
                        html.H4("Project status:", className="card-title"),
                        dcc.Checklist(      
                                    options=[
                                        {"label": "COMPLETED", "value": "COMPLETED", },
                                        {"label": "ON GOING", "value": "ON GOING"},
                                        {"label": "HOLDING", "value": "HOLDING"},
                                            ],
                                    value=["ON GOING"],
                                    id = "status_cell")
                        ]
                        
                            )
                    )

In [14]:
# DEFINES THE DYNAMIC TABLE FOR FILTERING THE LAB DATA
status_table = total_data[['CELL', 'TEST.ID', "P_N", "DUE.DATE", "TEST.DESCRIPTION", "TEST.STATUS", 'STIMATION', 'DURATION']]
#status_table['DUE.DATE'] = status_table['DUE.DATE'].dt.strftime('%m/%d/%Y')

dynamic_table = dash_table.DataTable(
                                    columns=[{"name": i, "id": i} for i in status_table.columns],
                                    page_size=10,
                                    fixed_rows={'headers': True},
                                    style_table={'height': '200px', 'overflowY': 'auto'},
                                    id= "total_data2",
                                    data=status_table.to_dict("records")                     
                                    )
                

table_card = dbc.Card(
        dbc.CardBody(
                dynamic_table      
                    )
)
#display(status_table)

In [28]:
#COUNT CELLS AVAILABILITY

total_stimation = total_data[['CELL', 'STIMATION', 'DURATION']]
date_df = total_data['DUE.DATE']

total_stimation_cell = total_stimation.groupby(['CELL']).sum().reset_index()
total_stimation_cell['STIMATION'] = total_stimation_cell['STIMATION'] / np.timedelta64(1, 'h')
total_stimation_cell['DURATION'] = total_stimation_cell['DURATION'] / np.timedelta64(1, 'h')

dynamic_table2 = dash_table.DataTable(
                                    columns=[{"name": i, "id": i} for i in total_stimation_cell.columns],
                                    page_size=10,
                                    fixed_rows={'headers': True},
                                    style_table={'height': '200px', 'overflowY': 'auto'},
                                    id= "time_data",
                                    data=total_stimation_cell.to_dict("records")                     
                                    )
                

table_card2 = dbc.Card(
        dbc.CardBody(
                dynamic_table2      
                    )
)

#display(total_stimation_cell)

In [16]:
#CELL AVAILABILITY TABLE

cells_availability = cells_calendar.drop(['WEEKDAY'], axis = 1)
cells_availability = cells_availability.drop(['DATE'], axis = 1)
cells_availability_40 = cells_availability['40'].sum()
cells_availability_60 = cells_availability['60'].sum()
cells_availability_110 = cells_availability['110'].sum()
cells_availability_resume = pd.DataFrame({'CELL': ['40', '60', '110'],
        'AVAILABILITY': [cells_availability_40, cells_availability_60, cells_availability_110]})

dynamic_table3 = dash_table.DataTable(
                                    columns=[{"name": i, "id": i} for i in cells_availability_resume.columns],
                                    #page_size=10,
                                    fixed_rows={'headers': True},
                                    style_table={'height': '200px', 'overflowY': 'auto'},
                                    id= "time_data_cells",
                                    data=cells_availability_resume.to_dict("records")                     
                                    )
                

table_card3 = dbc.Card(
        dbc.CardBody(
                dynamic_table3      
                    )
)

#display(cells_availability_resume)


In [24]:
#OPERATOR AVAILABILITY TABLE

operators_availability = operators_calendar.drop(['WEEKDAY'], axis = 1)
operators_availability = operators_availability.drop(['DATE'], axis = 1)
operators_availability_40 = operators_availability[40].sum()
operators_availability_60 = operators_availability[60].sum()
operators_availability_110 = operators_availability[110].sum()
operators_availability_resume = pd.DataFrame({'OPERATOR': ['GUERZONI', 'CARLINI', 'RESTANI'],
        'AVAILABILITY': [operators_availability_40, operators_availability_60, operators_availability_110]})

dynamic_table4 = dash_table.DataTable(
                                    columns=[{"name": i, "id": i} for i in operators_availability_resume.columns],
                                    #page_size=10,
                                    fixed_rows={'headers': True},
                                    style_table={'height': '200px', 'overflowY': 'auto'},
                                    id= "time_data_operators",
                                    data=operators_availability_resume.to_dict("records")                     
                                    )
                

table_card4 = dbc.Card(
        dbc.CardBody(
                dynamic_table4      
                    )
)

In [64]:
# HISTOGRAM CHART - TIME


total_time = pd.concat([cells_availability_resume, total_stimation_cell.drop(['CELL'], axis = 1)], axis=1, join='inner')


fig4 = go.Figure(data=[
    go.Bar(name='DURATION', x=total_time['CELL'], y=total_time['DURATION']),
    go.Bar(name='STIMATION', x=total_time['CELL'], y=total_time['STIMATION']),
    go.Bar(name='AVAILABILITY', x=total_time['CELL'], y=total_time['AVAILABILITY']),
])
fig4.update_layout(title_text='Time report - Cells')
fig4.update_layout(barmode='group')
fig4.show()




display(total_time)


,CELL,AVAILABILITY,STIMATION,DURATION
0,40,3080.0,8.0,0.0
1,60,2984.0,24.0,55.0
2,110,3032.0,280.0,111.8


In [29]:
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])


app.layout = [ navbar, dbc.Container(    

                                        html.Div(
                                            [
                                                dbc.Row(
                                                    [
                                                        dbc.Col(
                                                            [dbc.Row(date_card, style= {'background-color': '#ff8080'}),
                                                             dbc.Row(cell_card, style= {'background-color': '#ff8080'}),
                                                             dbc.Row(status_card, style= {'background-color': '#ff8080'})],
                                                            width = 2, style= {'background-color': '#ff8080'}),

                                                        dbc.Col(
                                                            [
                                                                dbc.Row(
                                                                    [
                                                                        dbc.Col(table_card, width = 8),
                                                                        dbc.Col(table_card2, width = 4)
                                                                     ]
                                                                ),
                                                                dbc.Row(
                                                                    [
                                                                    dbc.Col(table_card3, width = 4),
                                                                    dbc.Col(table_card4, width = 4),
                                                                    ]
                                                                    
                                                                ),
                                                                dbc.Row()
                                                            ], width = 10, style= {'background-color': '#ff8080'}),
                                                    ]
                                                )
                                            ], 
                                            style= {'background-color': '#ff8080', 'width': 1850, 'margin':0 })        
                                        , 
                                
                                fluid=True,
                                style={'display': 'flex'},
                                className='dashboard-container')    ]    

@app.callback(   
    Output("total_data2", 'data'),
    Input('selected_cell', 'value'),
    Input('status_cell', 'value'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date")
 )
            

def update_table(selected_cell, status_cell, start_date, end_date):

    filtered_df= status_table[ 
                                status_table["DUE.DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))&
                                status_table['CELL'].isin(selected_cell)  &
                                status_table['TEST.STATUS'].isin(status_cell)
                             ] 
    
    return filtered_df.to_dict("records") 

@app.callback(   
    Output("time_data", 'data'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date"),
    prevent_initial_call=True
    
 )
            

def update_table2(start_date, end_date):

    filtered_df2= total_stimation.join(date_df)
    filtered_df2 = filtered_df2[ filtered_df2["DUE.DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))]
    filtered_df2= filtered_df2.drop(['DUE.DATE'], axis=1)
    #pd.to_timedelta(filtered_df2['STIMATION'])
    #pd.to_timedelta(filtered_df2['DURATION'])
    filtered_df2['STIMATION'] = filtered_df2['STIMATION'] / np.timedelta64(1, 'h')
    filtered_df2['DURATION'] = filtered_df2['DURATION'] / np.timedelta64(1, 'h')
    
    filtered_df2=filtered_df2.groupby(['CELL']).sum().reset_index()
    
    return filtered_df2.to_dict("records") 

@app.callback(   
    Output("time_data_cells", 'data'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date"),
    prevent_initial_call=True
    
 )
            

def update_table3(start_date, end_date):

    filtered_df3 = cells_calendar.drop(['WEEKDAY'], axis = 1)
    filtered_df3 = filtered_df3[ filtered_df3["DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))]
    filtered_df3= filtered_df3.drop(['DATE'], axis=1)
    filtered_availability_40 = filtered_df3['40'].sum()
    filtered_availability_60 = filtered_df3['60'].sum()
    filtered_availability_110 = filtered_df3['110'].sum()
    filtered_availability_cells = pd.DataFrame({'CELL': ['40', '60', '110'],
        'AVAILABILITY': [filtered_availability_40, filtered_availability_60, filtered_availability_110]})

    
    return filtered_availability_cells.to_dict("records")

@app.callback(   
    Output("time_data_operators", 'data'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date"),
    prevent_initial_call=True
    
 )

def update_table4(start_date, end_date):

    filtered_df4 = operators_calendar.drop(['WEEKDAY'], axis = 1)
    filtered_df4 = filtered_df4[ filtered_df4["DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))]
    filtered_df4= filtered_df4.drop(['DATE'], axis=1)
    filtered_availability_operator_40 = filtered_df4[40].sum()
    filtered_availability_operator_60 = filtered_df4[60].sum()
    filtered_availability_operator_110 = filtered_df4[110].sum()
    filtered_availability_operators = pd.DataFrame({'OPERATOR': ['GUERZONI', 'CARLINI', 'RESTANI'],
        'AVAILABILITY': [filtered_availability_operator_40, filtered_availability_operator_60, filtered_availability_operator_110]})

    
    return filtered_availability_operators.to_dict("records")
#_________________________________________




if __name__ == '__main__':
    app.run_server(debug=False,dev_tools_ui=False,dev_tools_props_check=False, port=8051)
    




#http://127.0.0.1:8051/